In [1]:
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
df=pd.read_csv(r'J:\Data science project\Language Detection.csv')
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [3]:
x,y=df.iloc[:,0],df.iloc[:,1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8269,)
(2068,)
(8269,)
(2068,)


In [4]:
#Vectorizer and model fitting
Vectorizer=feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer='char')
pipe_lr_r13=pipeline.Pipeline([
    ('vectorizer',Vectorizer),
    ('clf',linear_model.LogisticRegression())
])

In [5]:
pipe_lr_r13.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [6]:
#model prediction
y_predict=pipe_lr_r13.predict(x_test)

In [7]:
#model evaluation
acc = (metrics.accuracy_score(y_test, y_predict))*100 
print(acc, '%')
matrix = metrics.confusion_matrix(y_test,y_predict)
print('Confusion matrix: \n', matrix)

98.25918762088975 %
Confusion matrix: 
 [[117   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  72   0   2   0   0   0   0   0   0   0   0   0   0   3   0   0]
 [  0   1 103   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   1 256   1   0   0   0   1   0   0   0   0   1   0   0   0]
 [  0   0   0   1 194   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   2   3   0   0 100   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  70   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 145   0   0   3   0   4   0   0   0]
 [  0   0   0   0   0   0   0   0   0  77   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 114   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 149   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 142   0   0   0   0]
 [  0   0   0   0   0   0   0   0   2

In [8]:
import pickle
# Persist model so that it can be used by different consumers 
lrFile=open('LRModel.pck1', 'wb')
pickle.dump(pipe_lr_r13, lrFile)
lrFile.close()
#Model Loading
global IrLangDetectModel
lrLangDetectFile = open('LRModel.pck1', 'rb')
lrLangDetectModel = pickle.load(lrLangDetectFile)
lrLangDetectFile.close()

In [9]:
def lang_detect(text):
    import numpy as np
    import string
    import re
    import pickle
    translate_table= dict((ord(char), None) for char in string.punctuation)
    global lrLangDetectModel
    lrLangDetectFile = open('LRModel.pck1', 'rb')
    lrLangDetectModel = pickle.load(lrLangDetectFile)
    lrLangDetectFile.close()
    text = "".join(text.split())
    text=text.lower()
    text =re.sub(r"\d+", "", text)
    text =text.translate (translate_table)
    pred = lrLangDetectModel.predict([text])
    prob = lrLangDetectModel.predict_proba([text])
    return pred [0]

In [10]:
lang_detect("Hello I just built my own language detection model")

'English'

In [11]:
lang_detect("Hallo, ich habe gerade mein eigenes Spracherkennungsmodell erstellt")

'German'

In [12]:
lang_detect("Bonjour, je viens de créer mon propre modèle de détection de langue")

'French'

In [13]:
lang_detect("Hola, acabo de construir mi propio modelo de detección de idioma")

'Spanish'